In [22]:
import os
import datetime
import pytz
import pickle
import numpy as np
import pandas as pd
import networkx as nx
from dotenv import find_dotenv, load_dotenv
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
from neo4j import GraphDatabase
from tqdm.notebook import tqdm

## Required Functions

This section definies various function for data retrieval from Neo4j database.

In [2]:
def get_packages(driver, date):
    """Queries packages from database

    Retrieves a list of packages that have a unique repository
    and have been created until the specified date.

    Args:
        driver: Neo4j database connection's driver.
        date: Date to filter packages creation.
    
    Returns:
        A pandas DataFrame containing the packages.
        Each row represents a package with the attributes
        'name', 'repo_name', 'repo_owner', and 'created'.
    """
    with driver.session(database='main') as session:
        query = """
                MATCH (pa:Package)-[r:DEVELOPED_AT]->(pr:Project)
                WITH pr, COUNT(r) AS num_pkgs
                WHERE num_pkgs = 1
                WITH pr
                MATCH (pa:Package)-[r:DEVELOPED_AT]->(pr)
                WHERE pa.created < DateTime($date)
                RETURN pa.name AS name,
                       pa.repo_name AS repo_name,
                       pa.repo_owner AS repo_owner,
                       toString(pa.created) AS created
                """
        
        results = session.run(query, date=date).data()
    
        packages = pd.DataFrame.from_dict(results)
        packages['created'] = pd.to_datetime(packages['created'])
        packages['observation'] = pd.to_datetime(date)
    
    return packages

In [3]:
def get_latest_version(driver, package, date):
    """Queries latest package version from database

    Retrieves the latest version for each package in list of
    names at observation date.

    Args:
        driver: Neo4j database connection's driver.
        package: Package name.
        date: Date of observation.
    
    Returns:
        A list of dictionaries with data.
    """
    with driver.session(database='main') as session:
        query = """
                OPTIONAL MATCH (p:Package { id: $package })-[:RELEASED]->(v:Version)
                WHERE v.created < DateTime($date)
                AND NOT v.number CONTAINS "-"
                RETURN p.name AS name,
                       v.id AS version_id,
                       v.number AS version,
                       v.license AS license,
                       toString(v.created) AS version_created
                ORDER BY v.created DESC
                LIMIT 1
                """
        return session.run(query, package=package, date=date).data()

In [4]:
def get_dependencies(driver, version_ids):
    """Queries version's dependencies from database.

    Retrieves the dependencies of for each package version
    in list of version IDs.

    Args:
        driver: Neo4j database connection's driver.
        version_ids: List of version IDs.
    
    Returns:
        A pandas DataFrame containing the all dependencies for
        each version ID. Each row represents an edge from the
        package towards its dependency with the attributes
        'source', 'target', 'type', and 'created.
    """
    with driver.session(database='main') as session:
        query = """
                UNWIND $versions AS version
                MATCH (v:Version { id: version })-[d:DEPENDS_ON]->(p:Package)
                RETURN v.package_id AS source,
                       p.id AS target,
                       d.requirements AS requirements,
                       toString(v.created) AS created
                """

        results = session.run(query, versions=version_ids).data()

        dependencies = pd.DataFrame.from_dict(results)
        dependencies['created'] = pd.to_datetime(dependencies['created'])

    return dependencies

In [16]:
def get_developers(driver, repository, start, end):
    """Queries participating developers for repositories from database.

    Retrieves the participating of for each repositories
    in list of IDs.

    Args:
        driver: Neo4j database connection's driver.
        repository: Repository IDs.
    
    Returns:
        A pandas DataFrame containing the participating users for
        a repository. Each row represents a developers with the attributes
        'XXX', 'XXX', 'XXX', and 'XXX'.
    """
    with driver.session(database='main') as session:
        query_comments = """
            MATCH (u:User)-[p:POSTED]->(c:Comment)-[*2]->(r:Repository)
            WHERE r.name = toString($repository)
            AND c.created >= datetime($start)
            AND c.created < datetime($end)
            AND u.type <> "Bot"
            RETURN DISTINCT u.login AS login
            """
                
        query_issues = """
            MATCH (u:User)-[a]->(:Issue)-[:RELATED_TO]->(r:Repository)
            WHERE r.name = toString($repository)
            AND a.created >= datetime($start)
            AND a.created < datetime($end)
            AND u.type <> "Bot"
            RETURN DISTINCT u.login AS login
            """

        query_pullreq = """
            MATCH (u:User)-[a]->(:PullRequest)-[:RELATED_TO]->(r:Repository)
            WHERE r.name = toString($repository)
            AND a.created >= datetime($start)
            AND a.created < datetime($end)
            AND u.type <> "Bot"
            RETURN DISTINCT u.login AS login
            """

        query_commits = """
            MATCH (u:User)-[a]->(:Commit)-[:RELATED_TO]->(r:Repository)
            WHERE r.name = toString($repository)
            AND a.created >= datetime($start)
            AND a.created < datetime($end)
            AND u.type <> "Bot"
            RETURN DISTINCT u.login AS login
            """

        results_comments = session.run(
            query_comments, repository=repository, start=start, end=end).data()
        results_issues = session.run(
            query_issues, repository=repository, start=start, end=end).data()
        results_pullreq = session.run(
            query_pullreq, repository=repository, start=start, end=end).data()
        results_commits = session.run(
            query_commits, repository=repository, start=start, end=end).data()

        comments = pd.DataFrame.from_dict(results_comments)        
        issues = pd.DataFrame.from_dict(results_issues)
        pullreqs = pd.DataFrame.from_dict(results_pullreq)
        commits = pd.DataFrame.from_dict(results_commits)

        developers = pd.concat([comments, issues, pullreqs, commits], axis=0)
        developers.drop_duplicates(subset=['login'], inplace=True)
        developers.reset_index(inplace=True)
        developers['repo_name'] = repository

    return developers.to_dict(orient='records')

## Database Connection

In [6]:
load_dotenv(find_dotenv())

# Get env variables
uri = os.getenv("NEO4J_URI")
user = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")

driver = GraphDatabase.driver(uri, auth=(user, password),
                              encrypted=False,
                              max_connection_lifetime=3600)

## Data Retrieval

### Set Observation Points

In [7]:
# Set periods for iteration
OBSERVATION_START = '2021-01-01'
DATE_START = '2021-04-01'  # Date of the first observation, React created at 
DATE_END = '2022-01-01'  # Date of last observation, full dataset 2020-03-01
PERIOD_LENGTH = relativedelta(months=3)  # Time between observations

period = DATE_START
periods = []
obs = 0
while period <= DATE_END:
    periods.append(period)
    next_period = (parse(period) + PERIOD_LENGTH).strftime("%Y-%m-%d")
    period = next_period
    obs += 1

observations = periods

[print(i+1, obs) for i, obs in enumerate(observations)];

1 2021-04-01
2 2021-07-01
3 2021-10-01
4 2022-01-01


### Create List of Packages

In [9]:
data = []
for obs in observations:
    packages = get_packages(driver, obs)

    latest_versions = []
    for package in tqdm(packages['name'].tolist()):
        latest_version = get_latest_version(driver, package, obs)
        latest_versions.append(latest_version[0])

    versions = pd.DataFrame.from_records(latest_versions)
    versions['version_created'] = pd.to_datetime(versions['version_created'])

    packages = packages.merge(versions, how="left", on=['name'])

    data.append(packages)

100%|██████████| 8208/8208 [00:10<00:00, 797.25it/s]


In [11]:
# TODO Need to focus on largest component again? Then, data from the step above needs to be filtered
# Create dependency networks for each observation
dependency_networks = []
with tqdm(total=len(observations)) as pbar:
    for i, obs in enumerate(observations):
        dependencies = get_dependencies(driver, data[i]['version_id'].tolist())

        # Keep edges between nodes in nodelist
        dependencies = dependencies[dependencies['target'].isin(data[i]['name'].tolist())]

        nodelist = data[i]['name'].tolist()
        edgelist = list(zip(dependencies['source'], dependencies['target']))

        G = nx.DiGraph()
        G.add_nodes_from(nodelist)
        G.add_edges_from(edgelist)
        dependency_networks.append(G)
        # nx.write_gpickle(G, '../data/dependency_network-{0}.pkl'.format(obs))
        # nx.write_gml(G, '../data/dependency_network-{0}.gml'.format(obs))
        pbar.update()
    pbar.close()

100%|██████████| 4/4 [00:05<00:00,  1.31s/it]


### Adjacency Matrices

In [77]:
# Latest observation makes nodelist to account for later joiners
full_packages = data[(len(observations) - 1)]
nodelist = full_packages['name'].tolist()

with tqdm(total=len(observations)) as pbar:
    for i, obs in enumerate(observations):
        adj = nx.to_pandas_adjacency(G, nodelist=nodelist)

        # Identify missing nodes at observation
        available_nodes = list(full_packages[full_packages['created'] < obs]['name'])
        missing_nodes = [item for item in nodelist if item not in available_nodes]

        # Change rows
        adj.loc[ missing_nodes , : ] = np.nan
        # Change columns
        adj.loc[ : , missing_nodes ] = np.nan

        adj = adj.astype('Int8')
        adj = adj.astype(str)
        adj.replace(to_replace='<NA>', value='NA', inplace=True)

        am = adj.to_numpy()
        np.savetxt('../data/net{0}.txt'.format(i+1), am, fmt='%s')
        np.savetxt('../data/am-{0}.txt'.format(obs), am, fmt='%s')

        # With structural zeros instead of NA for Goodness-of-Fit tests
        adj.replace(to_replace='NA', value='0', inplace=True)
        am = adj.to_numpy()
        np.savetxt('../data/net{0}-gof.txt'.format(i+1), am, fmt='%s')

        pbar.update()
    pbar.close()

100%|██████████| 4/4 [03:45<00:00, 56.50s/it]


## Affiliation Networks

In [23]:
# Create affiliation networks for each observation
affiliation_networks = []

with tqdm(total=len(observations)) as pbar:
    for i, obs in enumerate(observations):
        repo_names = data[i]['repo_name'].tolist()
        developers = []
        for repo in tqdm(repo_names, leave=False):
            if i == 0:
                devs = get_developers(driver, repo, OBSERVATION_START, obs)
            else:
                devs = get_developers(driver, repo, observations[i-1], observations[i])
            developers.extend(devs)

        df_devs = pd.DataFrame.from_records(developers)
        nodelist = df_devs['login'].unique().tolist()
        edgelist = list(zip(df_devs['login'], df_devs['repo_name']))

        G = nx.DiGraph()
        G.add_nodes_from(nodelist)
        G.add_nodes_from(repo_names)
        G.add_edges_from(edgelist)
        affiliation_networks.append(G)
        nx.write_gpickle(G, '../data/affiliation_network-{0}.pkl'.format(obs))
        nx.write_gml(G, '../data/affiliation_network-{0}.gml'.format(obs))
        pbar.update()
    pbar.close()

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/8108 [00:00<?, ?it/s]

  0%|          | 0/8141 [00:00<?, ?it/s]

  0%|          | 0/8180 [00:00<?, ?it/s]

  0%|          | 0/8208 [00:00<?, ?it/s]

### Composition Change

In [30]:
composition = df_created[['name', 'created']]
composition['appearance'] = 0

rev_obs = sorted(observations,  reverse=True)
for i, item in enumerate(rev_obs):
    obs = len(rev_obs) - i
    composition.loc[composition['created'] < item, 'appearance'] = int(obs)

arr = []
for i, row in composition.iterrows():
    curr = [int(row['appearance']), len(observations)]
    arr.append(curr)
    
comp_arr = np.array(arr)
np.savetxt('../rsiena/data/composition-changes.txt', comp_arr, fmt='%d')

### Dependency Types

In [24]:
for i, obs in enumerate(observations):
    G = nx.read_gpickle('data/graph-{0}.pkl'.format(obs))
    # Create subgraph based on final nodelist
    U = G.subgraph(nodelist).copy()
    # U = nx.subgraph(G, nodelist)

    # Convert list of tuples to list of lists for neo4j
    edgelist = [list(item) for item in list(U.edges())]

    # Query dependency types for all edges
    dependency_types = get_dependency_type(driver, edgelist, obs)

    # Change types to categorical codes
    dependency_types['type'].replace(to_replace='dependency', value=1, inplace=True)
    dependency_types['type'].replace(to_replace='devDependency', value=2, inplace=True)
    dependency_types['type'].replace(to_replace='peerDependency', value=3, inplace=True)

    # Add weights to edges according to code of dependency type
    for index, row in dependency_types.iterrows():
        U.edges[row['source'], row['target']]['weight'] = row['type']

    # Export
    adj = nx.to_pandas_adjacency(U, nodelist=nodelist, dtype='Int8')

    # Identify missing nodes at observation
    available_nodes = list(df_created[df_created['created'] < obs]['name'])
    missing_nodes = [item for item in nodelist if item not in available_nodes]

    # Change rows
    adj.loc[ missing_nodes , : ] = np.nan
    # Change columns
    adj.loc[ : , missing_nodes ] = np.nan

    adj = adj.astype('Int8')
    adj = adj.astype(str)
    adj.replace(to_replace='<NA>', value='NA', inplace=True)

    am = adj.to_numpy()
    np.savetxt('../rsiena/data/net{0}-type.txt'.format(i+1), am, fmt='%s')

    adj.replace(to_replace='NA', value='0', inplace=True)
    np.savetxt('../rsiena/data/net{0}-type-gof.txt'.format(i+1), am, fmt='%s')

### Update Behavior

In [46]:
updates = pd.DataFrame(nodelist, columns=['name'])
for obs in observations:
    _date_obs = datetime.datetime.strptime(obs, '%Y-%m-%d')
    timezone = pytz.timezone('UTC')
    date_obs = timezone.localize(_date_obs)

    latest_versions = get_latest_version(driver, nodelist, obs)

    data = {}
    for i, row in latest_versions.iterrows():
        delta = date_obs - row['created']
        data[row['name']] = delta
    
    _updates = pd.DataFrame.from_dict(data, orient='index', columns=[obs])
    updates = updates.merge(_updates, how='outer', right_index=True, left_on=['name'])

In [47]:
updates.describe()

,2016-01-01,2017-01-01,2018-01-01,2019-01-01
count,199,256,280,301
mean,106 days 08:01:24.643216080,183 days 21:15:55.984375,273 days 17:41:24.567857144,353 days 21:48:01.026578072
std,166 days 01:10:56.408254856,220 days 16:11:24.619665196,310 days 04:22:23.649908268,407 days 16:10:11.135340508
min,0 days 09:09:03,0 days 07:48:27,1 days 06:01:17,2 days 01:44:12
25%,22 days 00:19:51.500000,28 days 06:29:50,40 days 05:58:08.500000,37 days 10:07:19
50%,44 days 13:46:19,112 days 19:11:46,130 days 10:56:48,154 days 01:37:26
75%,133 days 19:37:29,243 days 07:01:34.750000,482 days 07:40:37.500000,572 days 07:52:17
max,1238 days 02:25:47,1604 days 02:25:47,1969 days 02:25:47,2334 days 02:25:47


In [48]:
updates_codes = updates.copy()
for i in range(len(observations)):

    updates_codes[observations[i]] = updates_codes[observations[i]].apply(
        lambda x: x.total_seconds()//(60*60*24)
    )

    for index, row in updates_codes.iterrows():
        # 2 Weeks
        if row[observations[i]] <= 14:
            updates_codes.loc[index, observations[i]] = 1
        # 1 Month
        elif row[observations[i]] <= 30:
            updates_codes.loc[index, observations[i]] = 2
        # 1 Quarter
        elif row[observations[i]] <= 90: 
            updates_codes.loc[index, observations[i]] = 3
        # Half a Year
        elif row[observations[i]] <= 180:
            updates_codes.loc[index, observations[i]] = 4
        # 1 Year
        elif row[observations[i]] <= 365:
            updates_codes.loc[index, observations[i]] = 5
        # More than 1 Year
        elif row[observations[i]] > 365:
            updates_codes.loc[index, observations[i]] = 6

updates_codes = updates_codes[observations].astype('Int8')
updates_codes = updates_codes[observations].astype(str)
updates_codes.replace(to_replace='<NA>', value='NA', inplace=True)
np.savetxt('../rsiena/data/updates-codes.txt', updates_codes[observations].values, fmt='%s')

### Licenses

In [49]:
licenses = pd.DataFrame(nodelist, columns=['name'])

for i, obs in enumerate(observations):
    latest_versions = get_latest_version(driver, nodelist, obs)
    latest_licenses = get_licenses(driver, list(latest_versions['_id']))
    licenses = licenses.merge(latest_licenses, how='outer', on=['name'])
    
licenses.columns = ['name'] + observations

In [50]:
licenses.head()

,name,2016-01-01,2017-01-01,2018-01-01,2019-01-01
0,@atlaskit/build-utils,NaN,NaN,Apache,Apache
1,@atlaskit/button,NaN,NaN,Apache,Apache
2,@atlaskit/docs,NaN,NaN,Apache,Apache
3,@atlaskit/icon,NaN,NaN,NULL,NULL
4,@atlaskit/theme,NaN,NaN,Apache,Apache


In [51]:
licenses_codes = licenses.copy()

licenses_codes.replace(to_replace='NULL', value=np.nan, inplace=True)
licenses_codes[observations] = licenses_codes[observations].apply(lambda col:pd.Categorical(col).codes)

licenses_codes = licenses_codes.astype(str)
licenses_codes.replace(to_replace='-1', value='NA', inplace=True)
licenses_codes.replace(to_replace='4', value='5', inplace=True)
licenses_codes.replace(to_replace='3', value='4', inplace=True)
licenses_codes.replace(to_replace='2', value='3', inplace=True)
licenses_codes.replace(to_replace='1', value='2', inplace=True)
licenses_codes.replace(to_replace='0', value='1', inplace=True)


np.savetxt('../rsiena/data/licenses-codes.txt',
           licenses_codes[observations].values, fmt='%s')

In [55]:
licenses_codes.head()

,name,2016-01-01,2017-01-01,2018-01-01,2019-01-01
0,@atlaskit/build-utils,NA,NA,1,1
1,@atlaskit/button,NA,NA,1,1
2,@atlaskit/docs,NA,NA,1,1
3,@atlaskit/icon,NA,NA,NA,NA
4,@atlaskit/theme,NA,NA,1,1


### Number of Releases

In [56]:
releases = pd.DataFrame(nodelist, columns=['name'])

for i in range(len(observations)):
    release_counts = get_releases(driver, nodelist, observations[i])
    release_counts.columns = ['name', observations[i]]
    releases = releases.merge(release_counts, how='outer', on=['name'])

In [57]:
releases.describe()

,2016-01-01,2017-01-01,2018-01-01,2019-01-01
count,199.000000,256.000000,280.000000,301.000000
mean,34.376884,40.207031,48.346429,55.534884
std,47.611047,48.636548,53.227159,63.142402
min,1.000000,1.000000,1.000000,1.000000
25%,9.000000,11.000000,16.000000,17.000000
50%,19.000000,23.000000,30.000000,36.000000
75%,42.000000,49.000000,58.000000,69.000000
max,455.000000,460.000000,468.000000,478.000000


In [58]:
releases_codes = releases.copy()

for i in range(len(observations)):
    for index, row in releases_codes.iterrows():
        if row[observations[i]] <= 5:
            releases_codes.loc[index, observations[i]] = 1
        elif row[observations[i]] <= 10:
            releases_codes.loc[index, observations[i]] = 2
        elif row[observations[i]] <= 20:
            releases_codes.loc[index, observations[i]] = 3
        elif row[observations[i]] <= 30:
            releases_codes.loc[index, observations[i]] = 4
        elif row[observations[i]] <= 40:
            releases_codes.loc[index, observations[i]] = 5
        elif row[observations[i]] <= 50: 
            releases_codes.loc[index, observations[i]] = 6
        elif row[observations[i]] <= 60:
            releases_codes.loc[index, observations[i]] = 7
        elif row[observations[i]] > 60:
            releases_codes.loc[index, observations[i]] = 8

releases_codes[observations] = releases_codes[observations].astype('Int8')
releases_codes[observations] = releases_codes[observations].astype(str)
releases_codes.replace(to_replace='<NA>', value='NA', inplace=True)
np.savetxt('../rsiena/data/releases-codes.txt',
           releases_codes[observations].values, fmt='%s')


### Age

In [59]:
ages = df_created[['name', 'created']]

for obs in observations:
    ages[obs] = ages['created'].apply(
        lambda x: datetime.datetime.strptime(obs, '%Y-%m-%d').date() - x.date())
    ages[obs] = round(ages[obs].dt.days/365, 0)
    ages[obs] = ages[obs].apply(lambda x : x if x > 0 else np.nan)

ages[observations] = ages[observations].astype('Int8')
ages[observations] = ages[observations].astype(str)

ages.replace(to_replace='<NA>', value='NA', inplace=True)
np.savetxt('../rsiena/data/ages.txt', ages[observations].values, fmt='%s')